<a href="https://colab.research.google.com/github/ShadyAK/GPT-CharLevel/blob/main/GPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 KB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 3.8 MB/s eta 0:00:00


In [5]:
#Downloading the model and saving it as saved_model
!wget -O saved_model https://github.com/ShadyAK/GPT-CharLevel/blob/main/saved_model?raw=true

--2023-04-05 12:11:17--  https://github.com/ShadyAK/GPT-CharLevel/blob/main/saved_model?raw=true
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/ShadyAK/GPT-CharLevel/raw/main/saved_model [following]
--2023-04-05 12:11:18--  https://github.com/ShadyAK/GPT-CharLevel/raw/main/saved_model
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/ShadyAK/GPT-CharLevel/main/saved_model [following]
--2023-04-05 12:11:18--  https://raw.githubusercontent.com/ShadyAK/GPT-CharLevel/main/saved_model
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK


In [6]:
import torch
import pickle
import re
import datasets
import torch

if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")
#Currently using CPU as GPU wasn't performing much better

In [7]:
# Getting data from HuggingFace's datasets package
from datasets import load_dataset

dataset = load_dataset("daily_dialog")


Generating train split:   0%|          | 0/11118 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Dataset daily_dialog downloaded and prepared to /root/.cache/huggingface/datasets/daily_dialog/default/1.0.0/1d0a58c7f2a4dab5ed9d01dbde8e55e0058e589ab81fce5c2df929ea810eabcd. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [8]:
train_data = dataset["train"]

def get_dialogue_from_dataset(data):
  result = ""
  for dialogue in data:
    result += " {}".format(".".join(dialogue['dialog']))
  return result

train_data = get_dialogue_from_dataset(train_data)
chars = sorted(list(set(train_data)))

In [9]:
#Char mappings
encode_chars = {chars[i] : i for i in range(len(chars))}
decode_chars = {i : chars[i] for i in range(len(chars))}
print(encode_chars)
print(decode_chars)
print(len(encode_chars))

{' ': 0, '!': 1, '"': 2, '#': 3, '$': 4, '%': 5, '&': 6, "'": 7, '(': 8, ')': 9, '*': 10, '+': 11, ',': 12, '-': 13, '.': 14, '/': 15, '0': 16, '1': 17, '2': 18, '3': 19, '4': 20, '5': 21, '6': 22, '7': 23, '8': 24, '9': 25, ':': 26, ';': 27, '=': 28, '?': 29, '@': 30, 'A': 31, 'B': 32, 'C': 33, 'D': 34, 'E': 35, 'F': 36, 'G': 37, 'H': 38, 'I': 39, 'J': 40, 'K': 41, 'L': 42, 'M': 43, 'N': 44, 'O': 45, 'P': 46, 'Q': 47, 'R': 48, 'S': 49, 'T': 50, 'U': 51, 'V': 52, 'W': 53, 'X': 54, 'Y': 55, 'Z': 56, '\\': 57, '_': 58, 'a': 59, 'b': 60, 'c': 61, 'd': 62, 'e': 63, 'f': 64, 'g': 65, 'h': 66, 'i': 67, 'j': 68, 'k': 69, 'l': 70, 'm': 71, 'n': 72, 'o': 73, 'p': 74, 'q': 75, 'r': 76, 's': 77, 't': 78, 'u': 79, 'v': 80, 'w': 81, 'x': 82, 'y': 83, 'z': 84, '~': 85, '\x7f': 86, '£': 87, '¥': 88, '°': 89, '–': 90, '—': 91, '‘': 92, '’': 93, '“': 94, '”': 95, '′': 96, '、': 97, '。': 98}
{0: ' ', 1: '!', 2: '"', 3: '#', 4: '$', 5: '%', 6: '&', 7: "'", 8: '(', 9: ')', 10: '*', 11: '+', 12: ',', 13: '-

In [10]:
# Encoder Decoder
def encode(s : str): return [encode_chars[c] for c in s]
def decode(s : list): return "".join([decode_chars[i] for i in s])

In [11]:
#Training Data Sample
train_data[:100]

' Say , Jim , how about going for a few beers after dinner ? . You know that is tempting but is reall'

In [12]:
train_data_tensor = torch.tensor(encode(train_data), dtype=torch.long)
print(train_data_tensor.shape, train_data_tensor.dtype)
print(train_data_tensor[:100])

torch.Size([5482281]) torch.int64
tensor([ 0, 49, 59, 83,  0, 12,  0, 40, 67, 71,  0, 12,  0, 66, 73, 81,  0, 59,
        60, 73, 79, 78,  0, 65, 73, 67, 72, 65,  0, 64, 73, 76,  0, 59,  0, 64,
        63, 81,  0, 60, 63, 63, 76, 77,  0, 59, 64, 78, 63, 76,  0, 62, 67, 72,
        72, 63, 76,  0, 29,  0, 14,  0, 55, 73, 79,  0, 69, 72, 73, 81,  0, 78,
        66, 59, 78,  0, 67, 77,  0, 78, 63, 71, 74, 78, 67, 72, 65,  0, 60, 79,
        78,  0, 67, 77,  0, 76, 63, 59, 70, 70])


In [13]:
val_data = get_dialogue_from_dataset(dataset['validation'])
val_data_tensor = torch.tensor(encode(val_data), dtype=torch.long)
print(val_data_tensor.shape, val_data_tensor.dtype)

torch.Size([506832]) torch.int64


In [14]:

def get_batch(data, batch_size, block_size):
  ix = torch.randint(len(data) - block_size, (batch_size,))
  x = torch.stack([data[i:i+block_size] for i in ix])  #following characters
  y = torch.stack([data[i+1:i+block_size+1] for i in ix]) #leading character
  return x,y

#xb, yb = get_batch(train_data_tensor, batch_size, block_size)

In [15]:
import torch.nn as nn 
from torch.nn import functional as F 
# To remove entropy every time weights are initialized to have unified results
torch.manual_seed(7)

In [16]:
class Head(nn.Module):

  def __init__(self, head_size, n_embd, block_size):
    super().__init__()
    self.key = nn.Linear(n_embd, head_size, bias = False)
    self.value = nn.Linear(n_embd, head_size, bias = False)
    self.query = nn.Linear(n_embd, head_size, bias = False)
    self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
    self.dropout = nn.Dropout(0.1)
  def forward(self, x):
    B, T, C = x.shape 
    k = self.key(x)
    q = self.query(x)

    wei = q @ k.transpose(-2,-1) / C**0.5 # B,T,T || dividing with sqrt of C to normalize the scores and not push towards extrimities
    wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
    wei = F.softmax(wei, dim=-1)
    wei = self.dropout(wei)
    v = self.value(x)
    out = wei @ v 
    return out


In [17]:
# Implementation of MultiHeadAttention
class MultiHeadAttention(nn.Module):
  def __init__(self, num_heads, head_size, n_embd, block_size):
    super().__init__()
    self.heads = nn.ModuleList([Head(head_size, n_embd, block_size) for _ in range(num_heads)])
    self.proj = nn.Linear(n_embd, n_embd)
    self.dropout = nn.Dropout(0.1)
  def forward(self, x):
    out = torch.cat([h(x) for h in self.heads], dim=-1)
    return self.dropout(self.proj(out))


In [54]:
# FeedForward NN layer
class FeedForward(nn.Module):
  def __init__(self, n_embd):
    super().__init__()
    self.net = nn.Sequential(
        nn.Linear(n_embd, 4 * n_embd), 
        nn.ReLU(), # Activation Function (commonly used in FF network)
        nn.Linear(4 * n_embd, n_embd),
        nn.Dropout(0.1)
    )
  
  def forward(self, x):
    return self.net(x)

In [19]:
# Transformer block
class Block(nn.Module):

  def __init__(self, num_heads, head_size, n_embd, block_size):
    super().__init__()
    head_size = n_embd // num_heads
    self.sa = MultiHeadAttention(num_heads, head_size, n_embd, block_size)
    self.ffwd = FeedForward(n_embd)
    self.ln1 = torch.nn.BatchNorm1d(block_size)
    self.ln2 = torch.nn.BatchNorm1d(block_size)
  def forward(self, x):
    
    x = x + self.sa(self.ln1(x)) 
    x = x + self.ffwd(self.ln2(x))
    return x

In [20]:
class BigramLanguageModelWithAttention(nn.Module):

  def __init__(self, vocab_size, n_embd, block_size, num_heads):
    super().__init__()
    self.vocab_size = vocab_size
    self.n_embd = n_embd
    self.block_size = block_size
    self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
    self.position_embedding_table = nn.Embedding(block_size, n_embd)
    self.sa_head = MultiHeadAttention(num_heads, n_embd//num_heads, n_embd, block_size)
    self.lm_head = nn.Linear(n_embd, vocab_size)
    self.ffwd = FeedForward(n_embd)
    self.blocks = nn.Sequential(
        Block(num_heads, n_embd//num_heads, n_embd, block_size),
        Block(num_heads, n_embd//num_heads, n_embd, block_size),
        Block(num_heads, n_embd//num_heads, n_embd, block_size)
    )
  
  def forward(self, idx, targets = None):
    B, T = idx.shape
    
    token_emb = self.token_embedding_table(idx)
    pos_emb = self.position_embedding_table(torch.arange(T))
    
    x = token_emb + pos_emb
    x = self.blocks(x)
    x = self.sa_head(x)
    x = self.ffwd(x)
    logits = self.lm_head(x)
    loss = None

    if targets is not None:
      B, T, C = logits.shape 
      logits = logits.view(B*T, C)
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits, targets)

    return logits, loss 

  def generate(self , idx, max_new_tokens):
    for _ in range(max_new_tokens):
      #To make sure idx does not exceed block size, position_embedding_table will run out of bound
      idx_cond = idx[:, -self.block_size:]
      logits, loss = self.forward(idx_cond) #(B,T,C)
      logits = logits[:, -1, :] # (B,C)
      probs = F.softmax(logits, dim=-1) # (B,C)
      idx_next = torch.multinomial(probs, num_samples=1) # (B,1)
      idx = torch.cat((idx, idx_next), dim=1) #(B,T+1)
    return idx  


## Model Summary  



```
BigramLanguageModelWithAttention(
  (token_embedding_table): Embedding(99, 128)
  (position_embedding_table): Embedding(10, 128)
  (sa_head): MultiHeadAttention(
    (heads): ModuleList(
      (0-3): 4 x Head(
        (key): Linear(in_features=128, out_features=32, bias=False)
        (value): Linear(in_features=128, out_features=32, bias=False)
        (query): Linear(in_features=128, out_features=32, bias=False)
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
    (proj): Linear(in_features=128, out_features=128, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (lm_head): Linear(in_features=128, out_features=99, bias=True)
  (ffwd): FeedForward(
    (net): Sequential(
      (0): Linear(in_features=128, out_features=512, bias=True)
      (1): ReLU()
      (2): Linear(in_features=512, out_features=128, bias=True)
      (3): Dropout(p=0.1, inplace=False)
    )
  )
  (blocks): Sequential(
    (0): Block(
      (sa): MultiHeadAttention(
        (heads): ModuleList(
          (0-3): 4 x Head(
            (key): Linear(in_features=128, out_features=32, bias=False)
            (value): Linear(in_features=128, out_features=32, bias=False)
            (query): Linear(in_features=128, out_features=32, bias=False)
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
        (proj): Linear(in_features=128, out_features=128, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (ffwd): FeedForward(
        (net): Sequential(
          (0): Linear(in_features=128, out_features=512, bias=True)
          (1): ReLU()
          (2): Linear(in_features=512, out_features=128, bias=True)
          (3): Dropout(p=0.1, inplace=False)
        )
      )
      (ln1): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (ln2): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): Block(
      (sa): MultiHeadAttention(
        (heads): ModuleList(
          (0-3): 4 x Head(
            (key): Linear(in_features=128, out_features=32, bias=False)
            (value): Linear(in_features=128, out_features=32, bias=False)
            (query): Linear(in_features=128, out_features=32, bias=False)
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
        (proj): Linear(in_features=128, out_features=128, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (ffwd): FeedForward(
        (net): Sequential(
          (0): Linear(in_features=128, out_features=512, bias=True)
          (1): ReLU()
          (2): Linear(in_features=512, out_features=128, bias=True)
          (3): Dropout(p=0.1, inplace=False)
        )
      )
      (ln1): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (ln2): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (2): Block(
      (sa): MultiHeadAttention(
        (heads): ModuleList(
          (0-3): 4 x Head(
            (key): Linear(in_features=128, out_features=32, bias=False)
            (value): Linear(in_features=128, out_features=32, bias=False)
            (query): Linear(in_features=128, out_features=32, bias=False)
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
        (proj): Linear(in_features=128, out_features=128, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (ffwd): FeedForward(
        (net): Sequential(
          (0): Linear(in_features=128, out_features=512, bias=True)
          (1): ReLU()
          (2): Linear(in_features=512, out_features=128, bias=True)
          (3): Dropout(p=0.1, inplace=False)
        )
      )
      (ln1): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (ln2): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
)
```



In [23]:
# Model Params
batch_size = 32
block_size = 10
max_iter = 500000
eval_interval = 1000
lr = 1e-3
eval_iters = 200
n_embd = 128
num_heads = 4
char_count = len(encode_chars)

In [33]:
@torch.no_grad() #it wont call backward() [disabling gradiant calculation] during inference which would save a lot of memory and computation
def estimate_loss(model):
  out = {}
  model.eval()
  for i, split in enumerate([train_data_tensor, val_data_tensor]):
    losses = torch.zeros(eval_iters)
    for k in range(eval_iters):
      X,Y = get_batch(split, batch_size, block_size)
      logits, loss = model(X,Y)
      losses[k] = loss.item()
    out["validation" if i else "train"] = losses.mean()
  model.train()
  return out 

In [34]:
def train_model(max_iter, eval_interval, model):
  optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
  for iter in range(max_iter):
    if iter % eval_interval == 0:
      losses = estimate_loss(model)
      print(f"step {iter}: train loss: {losses['train']} and val loss {losses['validation']}")
      xb, yb = get_batch(train_data_tensor, batch_size, block_size) 
      logits, loss = model(xb, yb)
      optimizer.zero_grad(set_to_none=True)
      loss.backward()
      optimizer.step()



In [48]:
# model loader
def load_model(path : str):
  model = BigramLanguageModelWithAttention(char_count, n_embd, block_size, num_heads)
  model.load_state_dict(torch.load(path))
  return model

In [28]:
# utility function for response generation
def generate_response(model, length: int, initial_text: str):
  indexes = torch.tensor(encode(initial_text))
  generated_sequence = model.generate(indexes.reshape(1, indexes.shape[0]), length)
  return generated_sequence, decode(generated_sequence[0].tolist())

In [ ]:
# For a new model
'''xb, yb = get_batch(train_data_tensor, batch_size, block_size)
attentionModel = BigramLanguageModelWithAttention(char_count, n_embd, block_size, num_heads)
logits, loss = attentionModel.forward(xb, yb)
print(logits.shape)
print(loss)
model = attentionModel'''

torch.Size([320, 99])
tensor(4.6313, grad_fn=<NllLossBackward0>)


In [51]:
def save_model(path, model):
  torch.save(model.state_dict(), path)

In [52]:
# Loading the trained model 
gpt_model = load_model('./saved_model')

In [ ]:
# Generating Response
# Note - length of initial_text in generate_response should be greater than 7 as check in batch_normalization is not there yet
#  due to which it normalizes the table by block_size everytime and if the respective dimension is less than 8 it won't be able to normalize which would
# be the case if the lenght of initial_text is < 8
sequence, decoded_seq = generate_response(gpt_model, 200, initial_text = "It's working") 
print(decoded_seq)

It's working that the roan wask is you just's creaswea soliassle you that harly lill onal spendsecoury the , cult I cally my prirs . . Hell a miflewy for for and ther windi that miserynsind ito with ? . You lorsr


## There's some kind of linguistically correct language generation here and there.
#### Considering we had 99 different chars to deal with the model is still somehow able to form words.

## **Example Generation - **  
I would like to more . . Aple . C.Shight veruts it upplion . . but that have ongt is Laby tatuthy ? . Vact proust sed do om re..5pe a momenbut the the Ihe more cafve quurling moome baells , sued . . hare to sip it is of ally . . . Okrehkir ron't muss arce ber ssorow ? . He for do . Ageftally loust warnd Byour . . I'm shapper pord is pan . . I what as ecousnoll apperost ! . ’ ll and ’ s this the bafter nit able ush ? . I'm pled uppully marn'm chell to magcallant boorlaut ? . The bels musm to thall canly't imempever your . Year for me need litN'reeg pollynot exor cafe to eispangayour bear hoally have it in ! I magreace ipsell Slhe vice greed know cable of for lasancy limplihere the peallifce aderse dic soimcurnt I ’ peull you naft seed of the furd . Preally , do'ke that's sideg the sake in .  falm ? . . ' fall to tever thaverights . . Agriscult Mast the som of on on to peat somill . Thank it'sed . I'm ladd . reparrig an af iles tays , came fwint opped chrumpust . Vust cjust bid to olace you on . . Aple Do cour decham are ’ cl famed the nust goarda the . . Durd , I lealy ’ ll sree poplasy of chanthise a thy am popevery ? . have you sour tighrite be the dig to it in the vacch . . Tihhhfist , have I cad . ' do beacthere my the call a neot ’ s do ull of . Noll my is . . Ok bare aGloccar coullding suppexity bot bovible-is in the eeve , it . . Rall , cour 

## Further Analysis

**HyperParameter Tuning -**  
We can utilise Grid Search technique to find the optimal hyperparameters like - batch_size, n_embds, n_heads

**Activation Functions and Loss Functions** -   
Different activation functions and loss functions could be tried out based on the end-goal. Using SGD instead of AdamW resulted in slower loss minisation for this current assignment.

**Generation Technique -**   
Character based language generation performs much worse than word level generation as context that words provide comparitively to characters is much relevant. Tokeniser's like [Tiktoken](https://github.com/openai/tiktoken) could be used for better context. 

    
      
     
      
# Next up is the Model visualization



In [96]:
# Model Architecture Generation
!pip install torchviz --quiet
from torchviz import make_dot
xb, yb = get_batch(train_data_tensor, batch_size, block_size) 
yhat = gpt_model(xb, yb)

make_dot(yhat, params=dict(list(gpt_model.named_parameters()))).render("gpt_model", format="png")


'gpt_model.png'

In [97]:
gpt_model.named_parameters()

<generator object Module.named_parameters at 0x7f730886cba0>

### Model Visualization

gpt_model.png